In [1]:
import pandas as pd
import numpy as np
from itertools import chain
import ast

Đọc dữ liệu từ file:

In [2]:
item_df = pd.read_csv('process_data_tft_each_item_csv.csv', usecols=[
                      'ID Match', 'Champion', 'List Name Item']).drop_duplicates().reset_index(drop=True)
item_df = item_df[['Champion', 'List Name Item']]
item_df

,Champion,List Name Item
0,"['TFT8_Gangplank', 'TFT8_Vi', 'TFT8_LeeSin', '...","[['TFT_Item_ZekesHerald'], ['TFT_Item_BrambleV..."
1,"['TFT8_Lux', 'TFT8_Annie', 'TFT8_Yuumi', 'TFT8...","[['TFT8_Item_HeartEmblemItem'], ['TFT_Item_Bra..."
2,"['TFT8_Galio', 'TFT8_Nasus', 'TFT8_Gangplank',...","[['TFT_Item_GargoyleStoneplate'], [], ['TFT_It..."
3,"['TFT8_Fiora', 'TFT8_Yasuo', 'TFT8_Vayne', 'TF...","[[], ['TFT8_Item_HackerEmblemItem'], ['TFT_Ite..."
4,"['TFT8_Lulu', 'TFT8_Annie', 'TFT8_Yuumi', 'TFT...","[[], ['TFT_Item_WarmogsArmor', 'TFT_Item_Titan..."
...,...,...
18091,"['TFT8_Blitzcrank', 'TFT8_Renekton', 'TFT8_Vi'...","[[], ['TFT_Item_ArchangelsStaff', 'TFT_Item_Je..."
18092,"['TFT8_Fiora', 'TFT8_Yasuo', 'TFT8_Vayne', 'TF...","[['TFT_Item_TitanicHydra'], ['TFT_Item_ChainVe..."
18093,"['TFT8_WuKong', 'TFT8_Jax', 'TFT8_Zoe', 'TFT8_...","[[], [], ['TFT8_Item_HeartEmblemItem'], ['TFT_..."
18094,"['TFT8_Kayle', 'TFT8_Gangplank', 'TFT8_Fiora',...","[[], [], [], [], ['TFT_Item_StatikkShiv', 'TFT..."


Đổi kiểu dữ liệu từ String sang List

In [3]:
def changeStringToList(col_name, df):
    for i in range(len(col_name)):
        temp = []
        for j in range(len(df[col_name[i]])):
            temp.append(ast.literal_eval(df[col_name[i]][j]))
        df[col_name[i]] = temp

In [4]:
col_name_type_list = ['Champion', 'List Name Item']
changeStringToList(col_name_type_list, item_df)

Tạo dataframe với mỗi tướng sẽ là trang bị gì:

In [5]:
list_champion = list(chain.from_iterable(item_df['Champion']))
list_item_of_champion = list(chain.from_iterable((item_df['List Name Item'])))

champion_df = pd.DataFrame(data=list_champion, columns=['Champion'])
item_of_champions_df = pd.DataFrame(data= list_item_of_champion, columns=['First', 'Second', 'Third', '0', '1', '2', '3']).iloc[:,:-4]

extracted_col = champion_df['Champion']

item_of_champions_df.insert(0, "Champion", extracted_col)
pos = ['First', 'Second', 'Third']
list_champion_unique = sorted(champion_df['Champion'].unique())


Với mỗi tướng sẽ tạo ra tập các trang bị được sử dụng với tỉ lệ là bao nhiêu:

In [10]:
list_item_unique = list(set(i for j in list_item_of_champion for i in j))
list_item_unique
data_model = pd.DataFrame(columns=list_champion_unique, index=list_item_unique)
for champ in list_champion_unique:
    first_item = item_of_champions_df[['Champion','First']].value_counts()[champ]
    second_item = item_of_champions_df[['Champion','Second']].value_counts()[champ]
    third_item = item_of_champions_df[['Champion','Third']].value_counts()[champ]  
    freq = first_item.add(second_item.add(third_item, fill_value=0), fill_value=0).sort_values(ascending=False) / first_item.add(second_item.add(third_item, fill_value=0), fill_value=0).sort_values(ascending=False).sum()
    data_model[champ] = freq


Chuyển đổi giá trị Nan thành 0 (ứng với những trang bị không được lên)

In [11]:
data_model.fillna(0)

,TFT8_Alistar,TFT8_Annie,TFT8_Aphelios,TFT8_Ashe,TFT8_AurelionSol,TFT8_BelVeth,TFT8_Blitzcrank,TFT8_Camille,TFT8_Chogath,TFT8_Draven,...,TFT8_Vayne,TFT8_Velkoz,TFT8_Vi,TFT8_Viego,TFT8_WuKong,TFT8_Yasuo,TFT8_Yuumi,TFT8_Zac,TFT8_Zed,TFT8_Zoe
TFT8_Item_MascotEmblemItem,0.000000,0.009747,0.000545,0.000909,0.001650,0.000889,0.005666,0.001135,0.007303,0.000000,...,0.000198,0.000863,0.009670,0.000767,0.023041,0.001868,0.000000,0.015912,0.000171,0.001040
TFT5_Item_FrozenHeartRadiant,0.000000,0.000157,0.000272,0.000000,0.000000,0.000000,0.000000,0.000000,0.000487,0.000000,...,0.000000,0.000000,0.000853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
TFT_Item_GuinsoosRageblade,0.002856,0.001100,0.129428,0.182727,0.028053,0.058693,0.000944,0.003973,0.005355,0.066216,...,0.112957,0.028473,0.003413,0.002686,0.000000,0.010743,0.005789,0.003978,0.050223,0.004784
TFT_Item_ChainVest,0.017849,0.004087,0.001362,0.000000,0.000413,0.000000,0.008499,0.002270,0.005842,0.000000,...,0.000198,0.000863,0.012514,0.000384,0.000000,0.003737,0.000579,0.012929,0.000514,0.000416
TFT8_Item_OxForceEmblemItem,0.000000,0.000000,0.000000,0.000000,0.003300,0.011561,0.004721,0.015891,0.001461,0.000000,...,0.000791,0.000431,0.002844,0.000000,0.000000,0.005605,0.000579,0.001492,0.013884,0.000832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TFT8_Item_DefenderEmblemItem,0.002142,0.000943,0.000000,0.000000,0.000413,0.000000,0.003777,0.000000,0.001947,0.001351,...,0.000000,0.000000,0.002275,0.000000,0.000000,0.000000,0.000193,0.000000,0.000000,0.000624
TFT8_Item_PranksterEmblemItem,0.019990,0.004873,0.001090,0.000909,0.000825,0.000889,0.000000,0.000000,0.002434,0.000000,...,0.000791,0.000863,0.005119,0.001535,0.000000,0.003737,0.000193,0.004973,0.001200,0.000000
TFT_Item_SeraphsEmbrace,0.002856,0.000314,0.007902,0.000000,0.036304,0.005780,0.002833,0.000000,0.008763,0.001351,...,0.004352,0.055651,0.003413,0.009593,0.000000,0.020084,0.240448,0.005967,0.001200,0.227953
TFT5_Item_InfinityEdgeRadiant,0.000000,0.000000,0.001090,0.000000,0.000000,0.000445,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000686,0.000000


In [12]:
data_model.to_csv('data_model_csv.csv')